Copyright (c) TranAn. All rights reserved.

Licensed under the MIT License.

Email: tranandeveloper@gmail.com

# Cài đặt package và kết nối với Workspace

***Trong notebook này bạn sẽ học được cách kết nối đến workspace của bạn và khởi tạo jobs cho workspace***

***Lưu ý***: Để có thể chạy được notebook này hãy chắc chắn rằng bạn đã khởi tạo được workspace. Nếu không hãy xem lại bài đầu tiên để khởi tạo.

**Yêu cầu**: Đã cài đặt đầy đủ dependencies. Đối với Azure ml yêu cầu python > 3.7. Chưa cài đặt có thể cài qua lệnh

```
pip install azure-ai-ml
```

> Hãy thay key value cho subcriptions của bạn

In [5]:
import os

subscription_id = os.getenv("SUBSCRIPTION_ID", default="<my-subscription-id>")
resource_group = os.getenv("RESOURCE_GROUP", default="<my-resource-group>")
workspace_name = os.getenv("WORKSPACE_NAME", default="<my-workspace-name>")
workspace_region = os.getenv("WORKSPACE_REGION", default="eastus2")

---
# Kết nối với Azure Machine Learning Client
- credential: Là gói xác thực hiện tại đang để Default

Hãy chắc chắn rằng bạn đã xác thực cả AzureCLI ```azd auth login```

In [6]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

credential = DefaultAzureCredential()

ml_client = MLClient(
    credential=credential,
    subscription_id=subscription_id,
    resource_group_name=resource_group,
    workspace_name=workspace_name,
)

# Tạo compute
---

**Lý do**: Tại sao phải tạo computer? Hiểu đơn giản ở đây dù bạn đã khởi tạo dịch vụ AzureML nhưng đó mới chỉ là bạn đăng ký để sử dụng. Các tài nguyên sẽ được quản lý trong dịch vụ đó. Khi bạn muốn train, predict, xử lý dữ liệu,... bạn cần tài nguyên hoặc máy chủ computer. Việc cần thiết ở đây là bạn phải lựa chọn và khởi tạo cấu hình dịch vụ computer mà bạn mong muốn để chạy yêu cầu của bạn.

**Các bước khởi tạo**
1. Từ màn hình chính của AzureML Studio lựa chọn Manage/computer
2. Ở đây có nhiều dịch vụ computer phù hợp cho từng tác vụ khác nhau. (ở đây chúng ta đang thử nghiệm đào tạo và develop nên tôi khuyên bạn chọn compute instance)
3. Lựa chọn cài đặt cấu hình mong muốn và ấn nút create


In [7]:
COMPUTE_NAME = os.getenv("COMPUTE_NAME", default="<Name>")

## Khởi tạo compute bằng Azure CLI
---

***Lưu ý***: Hãy chắc chắn rằng bạn đã cài đặt AzureCLI trên máy của mình hoặc sử dụng Azure Cloud Shell. Cho dù bạn dùng platform nào thì các chức năng nó cũng hoàn toàn giống nhau. Sau khi cài đặt AzureCLI bạn cần phải cài đặt AzureML extensions để có thể quản lý được Azure Machine Learning resources của bạn.

1. Cài đặt AzureML extension qua lệnh sau:
    ```
    az extension add -n ml -y
    ```
    Sau khi cài đặt thành công bạn có thể thử dụng lệnh ```-h``` để kiểm tra xem extension đã được cài đặt hay chưa và lấy danh sách hỗ trợ tính năng của extentions này.

2. Sau khi cài đặt bạn có thể tương tác với Azure Machine Learning Workspace bằng lệnh cmd. Từng lệnh cmd đều có tiền tố ```az ml``` đứng trước để xác định rằng đang sử dụng AzureML extension.

***Ví dụ***: Bạn có thể khởi tạo mục tiêu compute cluster của mình với lệnh cmd dưới đây: 
```
az ml compute create --name aml-cluster --size <name virtual machine> --min-instances 0 --max-instances 5 --type AmlCompute --resource-group <name-your-resource-group> --workspace-name <name-your-workspace>
```
**Hoặc**: Bạn hoàn toàn có thể sử dụng tệp yaml để cấu hình cho compute của mình:

In [13]:
%%writefile compute_cluster.yml
$schema: https://azuremlschemas.azureedge.net/latest/amlCompute.schema.json 
name: aml-cluster
type: amlcompute
size: <name-of-vm-size>
min_instances: 0
max_instances: 5

Overwriting compute_cluster.yml


Sau đó bạn chạy lệnh dưới đây để build compute của bạn:
```
az ml compute create --file <name-file-yml>.yml --resource-group <name-your-resource-group> --workspace-name <name-your-workspace>
```

***Lưu ý***: Phải chạy đúng path nơi để file yml
> Bạn có thể tìm thấy thêm cài đặt file yml <a href="https://learn.microsoft.com/en-us/azure/machine-learning/reference-yaml-overview?view=azureml-api-2">tại đây</a>

---
# Tạo môi trường cho job

Để chạy một job trong AzureML bạn cần đến một môi trường. Có thể tìm kiếm môi trường phù hợp trong Environmentstại cổng AzureMLStudio

Ở notebook này tôi sẽ ví dụ sử dụng môi trường ```AzureML-sklearn-1.0-ubuntu20.04-py38-cpu@latest``` vì nó có chứa các thư viện cơ bản cần thiết cho ML như (python, MLflow, numpy, pip, etc).

# Khởi tạo Job để huấn luyện

In [9]:
# Khởi tạo vùng chứa cho job
import os

train_src_dir = "./src"
os.makedirs(train_src_dir, exist_ok=True)

In [10]:
%%writefile {train_src_dir}/main.py
import os
import argparse
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

def main():
    """Main function of the script."""

    # input and output arguments
    parser = argparse.ArgumentParser()
    parser.add_argument("--data", type=str, help="path to input data")
    parser.add_argument("--test_train_ratio", type=float, required=False, default=0.25)
    parser.add_argument("--n_estimators", required=False, default=100, type=int)
    parser.add_argument("--learning_rate", required=False, default=0.1, type=float)
    parser.add_argument("--registered_model_name", type=str, help="model name")
    args = parser.parse_args()
   
    # start Logging
    mlflow.start_run()

    # enable autologging
    mlflow.sklearn.autolog()

    ###################
    #<prepare the data>
    ###################
    print(" ".join(f"{k}={v}" for k, v in vars(args).items()))

    print("input data:", args.data)
    
    credit_df = pd.read_csv(args.data, header=1, index_col=0)

    mlflow.log_metric("num_samples", credit_df.shape[0])
    mlflow.log_metric("num_features", credit_df.shape[1] - 1)

    train_df, test_df = train_test_split(
        credit_df,
        test_size=args.test_train_ratio,
    )
    ###################
    #</prepare the data>
    ###################

    ##################
    #<train the model>
    ##################
    # extracting the label column
    y_train = train_df.pop("default payment next month")

    # convert the dataframe values to array
    X_train = train_df.values

    # extracting the label column
    y_test = test_df.pop("default payment next month")

    # convert the dataframe values to array
    X_test = test_df.values

    print(f"Training with data of shape {X_train.shape}")

    clf = GradientBoostingClassifier(
        n_estimators=args.n_estimators, learning_rate=args.learning_rate
    )
    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)

    print(classification_report(y_test, y_pred))

    ##################
    #</train the model>
    ##################

    ##########################
    #<save and register model>
    ##########################
    # registering the model to the workspace
    print("Registering the model via MLFlow")
    mlflow.sklearn.log_model(
        sk_model=clf,
        registered_model_name=args.registered_model_name,
        artifact_path=args.registered_model_name,
    )

    # saving the model to a file
    mlflow.sklearn.save_model(
        sk_model=clf,
        path=os.path.join(args.registered_model_name, "trained_model"),
    )
    
    # stop Logging
    mlflow.end_run()

if __name__ == "__main__":
    main()

Overwriting ./src/main.py


In [11]:
# import the libraries
from azure.ai.ml import command
from azure.ai.ml import Input

# name the model you registered earlier in the training script
registered_model_name = "credit_defaults_model"

# configure the command job
job = command(
    inputs=dict(
        # uri_file refers to a specific file as a data asset
        data=Input(
            type="uri_file",
            path="https://azuremlexamples.blob.core.windows.net/datasets/credit_card/default%20of%20credit%20card%20clients.csv",
        ),
        test_train_ratio=0.2,  # input variable in main.py
        learning_rate=0.25,  # input variable in main.py
        registered_model_name=registered_model_name,  # input variable in main.py
    ),
    code="./src/",  # location of source code
    # The inputs/outputs are accessible in the command via the ${{ ... }} notation
    command="python main.py --data ${{inputs.data}} --test_train_ratio ${{inputs.test_train_ratio}} --learning_rate ${{inputs.learning_rate}} --registered_model_name ${{inputs.registered_model_name}}",
    # This is the ready-made environment you are using
    environment="AzureML-sklearn-1.0-ubuntu20.04-py38-cpu@latest",
    # This is the compute you created earlier. You can alternatively remove this line to use serverless compute to run the job
    compute=COMPUTE_NAME,
    # An experiment is a container for all the iterations one does on a certain project. All the jobs submitted under the same experiment name would be listed next to each other in Azure ML studio.
    experiment_name="train_model_credit_default_prediction",
    display_name="credit_default_prediction",
)

In [ ]:
# submit the command job
ml_client.create_or_update(job)

> Code ví dụ được lấy tại Microsoft SDK. Truy cập vào AzureML sdk để lấy thêm dữ liệu